In [2]:
import tensorflow as tf
import numpy      as np
import pandas     as pd
import seaborn    as sns
from sklearn.model_selection   import train_test_split
import matplotlib.pyplot as plt
import missingno as msno

In [3]:
airPressure_area_frm = pd.read_csv('../content/airPressure_area_0708.csv', encoding='cp949')
humidity_area_frm = pd.read_csv('../content/humidity_area_0708.csv', encoding='cp949')
rain_area_frm = pd.read_csv('../content/rain_area_0708.csv', encoding='cp949')
sun_area_frm = pd.read_csv('../content/sun_area_0708.csv', encoding='cp949')
temp_area_frm = pd.read_csv('../content/temp_area_0708.csv', encoding='cp949')

In [4]:
# Unnamed: 0 index 삭제
del airPressure_area_frm["Unnamed: 0"]
del humidity_area_frm["Unnamed: 0"]
del rain_area_frm["Unnamed: 0"]
del sun_area_frm["Unnamed: 0"]
del temp_area_frm["Unnamed: 0"]

#### 지점 드랍 후 지역 일시 *groupby*

In [5]:
# 기압 지점 드랍 후 지역, 일시 groupby
airPressure_area_grp = airPressure_area_frm.drop('stn_id',axis=1).groupby(['area',"tma"], as_index = False).mean()

# 습도 지점 드랍 후 지역, 일시 groupby
humidity_area_grp = humidity_area_frm.drop('stn_id',axis=1).groupby(['area',"tma"], as_index = False).mean()

# 강수량 지점 드랍 후 지역, 일시 groupby
rain_area_grp = rain_area_frm.drop('stn_id',axis=1).groupby(['area',"tma"], as_index = False).mean()

# 일사량 지점 드랍 후 지역, 일시 groupby
sun_area_grp = sun_area_frm.drop('stn_id',axis=1).groupby(['area',"tma"], as_index = False).mean()

# 기온 지점 드랍 후 지역, 일시 groupby
temp_area_grp = temp_area_frm.drop('stn_id',axis=1).groupby(['area',"tma"], as_index = False).mean()

In [6]:
# airPressure, humidity, rain, sun, temp
# row 31586으로 딱 떨어짐.                        
# 얘네 가지고 결측치 채움.                        

In [7]:
# # 로직 : airPressure_area_frm의 결측치 부분의 일자와 지역이 airPressure_area_grp의 부분과 같으면 airPressure_area_grp의 값으로 대체.
# for i in range(1246362):
#   if (airPressure_area_frm["avg_ps"][i].isnull == True) and (airPressure_area_frm["tma"] == airPressure_area_grp["tma"] and (airPressure_area_frm["area"] == airPressure_area_grp["area"])):
#     airPressure_area_frm["avg_ps"] = airPressure_area_grp["avg_ps"]

# 집 가서 다시 보겠습니다.

In [8]:
# "csv파일"_area_grp.head()

#### 결측치 지역평균 및 max-min처리

In [9]:
# rain 결측치 지역평균으로 대체 -> 기상데이터를 지역 기준으로 보기때문에 지역 평균으로 대체
rain_avg_by_date = rain_area_frm.pivot_table(index = 'tma', values = 'sum_rn', aggfunc='mean')

rain_fill = rain_area_frm.apply(lambda frm :  rain_avg_by_date.loc[frm['tma'], 'sum_rn'] if (frm['sum_rn'] != frm['sum_rn']) 
                  else frm['sum_rn'], axis=1)

rain_area_frm['sum_rn'] = rain_fill

In [10]:
# temp_max 결측치 지역평균으로 대체 -> 기상데이터를 지역 기준으로 보기때문에 지역 평균으로 대체
temp_max_by_date = temp_area_frm.pivot_table(index = 'tma', values = 'max_ta', aggfunc='mean')

temp_max_fill = temp_area_frm.apply(lambda frm :  temp_max_by_date.loc[frm['tma'], 'max_ta'] if (frm['max_ta'] != frm['max_ta']) 
                  else frm['max_ta'], axis=1)

temp_area_frm['max_ta'] = temp_max_fill

In [11]:
# temp_min 결측치 지역평균으로 대체 -> 기상데이터를 지역 기준으로 보기때문에 지역 평균으로 대체
temp_min_by_date = temp_area_frm.pivot_table(index = 'tma', values = 'min_ta', aggfunc='mean')

temp_min_fill = temp_area_frm.apply(lambda frm :  temp_min_by_date.loc[frm['tma'], 'min_ta'] if (frm['min_ta'] != frm['min_ta']) 
                  else frm['min_ta'], axis=1)
temp_area_frm['min_ta'] = temp_min_fill

In [13]:
# temp_avg 결측치 지역평균으로 대체 -> 한시간마다 측정한 기온값들 다 더해서 24로 나눈것이 temp avg값이라 대체
temp_avg_by_date = temp_area_frm.pivot_table(index = 'tma', values = 'avg_ta', aggfunc='mean')

temp_avg_fill = temp_area_frm.apply(lambda frm :  temp_avg_by_date.loc[frm['tma'], 'avg_ta'] if (frm['avg_ta'] != frm['avg_ta']) 
                  else frm['avg_ta'], axis=1)
temp_area_frm['avg_ta'] = temp_avg_fill

In [14]:
#결측값 확인
print("airPressure_area_frm.isnull")
print(airPressure_area_frm.isnull().sum())
print("*"*50)
print()
print("humidity_area_frm.isnull")
print(humidity_area_frm.isnull().sum())
print("*"*50)
print()
print("rain_area_frm.isnull")
print(rain_area_frm.isnull().sum())
print("*"*50)
print()
print("sun_area_frm.isnull")
print(sun_area_frm.isnull().sum())
print("*"*50)
print()
print("temp_area_frm.isnull")
print(temp_area_frm.isnull().sum())

airPressure_area_frm.isnull
tma            0
stn_id         0
avg_ps    780987
max_ps    752147
min_ps    753504
area           0
dtype: int64
**************************************************

humidity_area_frm.isnull
tma             0
stn_id          0
avg_rhm    575031
min_rhm    554695
area            0
dtype: int64
**************************************************

rain_area_frm.isnull
tma       0
stn_id    0
sum_rn    0
area      0
dtype: int64
**************************************************

sun_area_frm.isnull
tma        0
stn_id     0
sum_gsr    0
area       0
dtype: int64
**************************************************

temp_area_frm.isnull
tma       0
stn_id    0
avg_ta    0
max_ta    0
min_ta    0
area      0
dtype: int64
